# Idea
The idea is that we predict rotated linear dimension position from lines and text from the drawing.

# Thoughts
Basic logic is we split dataset to input lines or texts positions StartPoint, EndPoint, Position XYZ and predict dimension extension line poistion XLine1Point, XLine2Point XYZ.

We going to group samples by FileId. That is each sample will contain variable length data (attributes of variable number of  lines and text) and variable output data (variable number of dimensions).

I intend to [use RNN for it](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)

In [ ]:
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient
from processing import *

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
test_data = pd.read_pickle('test_dataset.pickle')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9180 entries, 867872 to 6212
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   ClassName      9180 non-null   category
 1   FileId         9180 non-null   category
 2   XLine1Point.X  466 non-null    float64 
 3   XLine1Point.Y  466 non-null    float64 
 4   XLine1Point.Z  466 non-null    float64 
 5   XLine2Point.X  466 non-null    float64 
 6   XLine2Point.Y  466 non-null    float64 
 7   XLine2Point.Z  466 non-null    float64 
 8   StartPoint.X   7748 non-null   float64 
 9   StartPoint.Y   7748 non-null   float64 
 10  StartPoint.Z   7748 non-null   float64 
 11  EndPoint.X     7748 non-null   float64 
 12  EndPoint.Y     7748 non-null   float64 
 13  EndPoint.Z     7748 non-null   float64 
 14  Position.X     966 non-null    float64 
 15  Position.Y     966 non-null    float64 
 16  Position.Z     966 non-null    float64 
dtypes: category(2), float64(15)


# Dataset implementation

In [11]:
# https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
import torch
from torch import nn
from torch.utils.data import Dataset, SubsetRandomSampler

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    
device

/home/mk/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [12]:
x_columns = [
    #'ClassName', 
    'StartPoint.X', 'StartPoint.Y', 'StartPoint.Z',
    'EndPoint.X', 'EndPoint.Y', 'EndPoint.Z',
    'Position.X', 'Position.Y', 'Position.Z']

y_columns = [
    #'ClassName', 
    'XLine1Point.X', 'XLine1Point.Y','XLine1Point.Z', 
    'XLine2Point.X', 'XLine2Point.Y', 'XLine2Point.Z']

In [13]:

class EntityDataset(Dataset):
    def __init__(self, pandasData):
        self.data = pandasData
        self.groupped = data.groupby('FileId')
        self.keys = list(self.groupped.groups.keys() )
        
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self, index):
        x = self.groupped.get_group(self.keys[index])[x_columns]
        y = self.groupped.get_group(self.keys[index])[y_columns]
        
        return torch.tensor(np.array(x.values)), torch.tensor(np.array(y.values))

In [14]:
entities = EntityDataset(test_data)
c,d = entities[3]
print(c.shape, d.shape)

torch.Size([1265, 9]) torch.Size([1265, 6])


In [19]:
batch_size = 1

data_len = len(entities)

validation_fraction = 0.2
test_fraction       = 0.3

val_split  = int(np.floor(validation_fraction * data_len))
test_split = int(np.floor(test_fraction * data_len))
indices = list(range(data_len))
np.random.seed(228)

np.random.shuffle(indices)

val_indices   = indices[:val_split]
test_indices  = indices[val_split:test_split]
train_indices = indices[test_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler   = SubsetRandomSampler(val_indices)
test_sampler  = SubsetRandomSampler(test_indices)

# https://stackoverflow.com/questions/64586575/adding-class-objects-to-pytorch-dataloader-batch-must-contain-tensors
def custom_collate(sample):
    print(len(sample))
    x,y = sample[0]
    # print(x.shape,y.shape)
    
    return x,y

train_loader = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = train_sampler, collate_fn=custom_collate)
val_loader   = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = val_sampler)
test_loader  = torch.utils.data.DataLoader(entities, batch_size = batch_size, sampler = test_sampler)

In [20]:
test_data

,_id,Color,Layer,LineWeight,Linetype,ObjectId,ClassName,EndPoint,Length,StartPoint,...,StartPoint.Y,StartPoint.Z,EndPoint.ClassName,EndPoint.X,EndPoint.Y,EndPoint.Z,Position.ClassName,Position.X,Position.Y,Position.Z
578960,5fd1d6bc9002fcba81c6816d,BYLAYER,Defpoints,ByLayer,ByLayer,2273423817536,AcDbLine,"{'ClassName': 'Point3D', 'X': 683037.410150086...",3200.000000,"{'ClassName': 'Point3D', 'X': 683688.137187546...",...,-119664.338336,0.000000,Point3D,683037.410150,-122797.476432,0.000000,NaN,NaN,NaN,NaN
765464,5fd1e5a360505b08aa6afb57,white,АР_Разрез тонкая,ByLayer,ByLayer,2282481202672,AcDbLine,"{'ClassName': 'Point3D', 'X': 68925.2317877975...",1086.029185,"{'ClassName': 'Point3D', 'X': 67947.3229365530...",...,-127766.850889,0.000005,Point3D,68925.231788,-127294.459453,0.000003,NaN,NaN,NaN,NaN
502827,5fd1d53b9002fcba81bc8442,white,размеры,ByBlock,Continuous,2273423855488,AcDbLine,"{'ClassName': 'Point3D', 'X': 485129.259012423...",1249.997500,"{'ClassName': 'Point3D', 'X': 486379.256512424...",...,241842.508559,0.000000,Point3D,485129.259012,241842.508559,0.000000,NaN,NaN,NaN,NaN
627246,5fd1ddfd60505b08aa4fa47d,blue,DIM-TXT,ByLayer,BYLAYER,2282125447456,AcDbLine,"{'ClassName': 'Point3D', 'X': 196.671818798365...",2.999971,"{'ClassName': 'Point3D', 'X': 198.793118798365...",...,503.491026,0.000000,Point3D,196.671819,501.369726,0.000000,NaN,NaN,NaN,NaN
193772,5fd0cd14f44995d7a5aed2b0,BYLAYER,0,ByLayer,ByLayer,2143517767856,AcDbLine,"{'ClassName': 'Point3D', 'X': 5390.50733040096...",1.499996,"{'ClassName': 'Point3D', 'X': 5389.00733485290...",...,789.484476,0.000000,Point3D,5390.507330,789.484476,0.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391680,5fd0d07af44995d7a5b45209,green,0,ByLayer,BYLAYER,2144692062960,AcDbLine,"{'ClassName': 'Point3D', 'X': 119.819777312149...",5.491812,"{'ClassName': 'Point3D', 'X': 116.819777312149...",...,532.976630,0.000000,Point3D,119.819777,537.576630,0.000000,NaN,NaN,NaN,NaN
670787,5fd1e02960505b08aa57da53,white,A-Wall-G,LineWeight009,Continuous,2282133263856,AcDbLine,"{'ClassName': 'Point3D', 'X': -62588.638750925...",169.705627,"{'ClassName': 'Point3D', 'X': -62708.638750925...",...,-79279.626081,1200.000000,Point3D,-62588.638751,-79159.626081,1200.000000,NaN,NaN,NaN,NaN
708181,5fd1e03660505b08aa587c80,white,A-Wall-G,LineWeight009,Continuous,2282505973360,AcDbLine,"{'ClassName': 'Point3D', 'X': -129436.25179348...",70.710678,"{'ClassName': 'Point3D', 'X': -129386.25179348...",...,-90705.191159,3600.000000,Point3D,-129436.251793,-90655.191159,3600.000000,NaN,NaN,NaN,NaN
359753,5fd0cf70f44995d7a5b28835,red,AS,ByLayer,CONTINUOUS,2143356952960,AcDbLine,"{'ClassName': 'Point3D', 'X': 210.430862464636...",0.000000,"{'ClassName': 'Point3D', 'X': 210.430862464636...",...,225.902133,0.000000,Point3D,210.430862,225.902133,0.000000,NaN,NaN,NaN,NaN


In [21]:
(a,b) = next(iter(train_loader))

print(a.shape,b.shape)

1
torch.Size([148, 9]) torch.Size([148, 6])


# Model and training


In [22]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first = True)
        self.output_layer = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
        # print('x:', x.shape)
        batch_size = x.shape[0]
        hidden = self.init_hidden(batch_size)
        #print(x.shape, hidden.shape)
        out, hidden = self.rnn(x, hidden)
        
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    def init_hidden(self, batch_size):
        # print(batch_size)
        hidden = torch.zeros(batch_size, self.n_layers,  self.hidden_dim)
        return hidden

In [24]:
# https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
# shape coordination
def test_rnn_shapes():
    in_size_features = 9
    out_size_featrues = 6
    hidden_size = 12
    batch_l = 65
    num_files = 166
    num_rnn_layers = 2
    rnn = torch.nn.RNN(input_size = in_size_features, hidden_size = hidden_size, num_layers = num_rnn_layers)
    inpt = torch.randn(num_files, batch_l, in_size_features)
    h0 = torch.randn(num_rnn_layers, batch_l, hidden_size)
    out, hn= rnn(inpt, h0)
    print ('out.shape', out.shape, 'hn.shape', hn.shape)
test_rnn_shapes()

out.shape torch.Size([166, 65, 12]) hn.shape torch.Size([2, 65, 12])


In [350]:
for idx, (inp, outp) in enumerate(train_loader):
    print("input shape:", inp.shape)
    out, hidden = rnn(inp)
    print("output shape", out.shape)
    break

1
input shape: torch.Size([671, 9])


RuntimeError: input must have 3 dimensions, got 2

In [280]:
def plot_history(loss_history, train_history, val_history):
    plt.ylabel('Accuracy @ epoch')
    
    train, = plt.plot(train_history)
    train.set_label("train")
    validation, = plt.plot(val_history)
    validation.set_label("validation")

    plt.legend()
    plt.show()

In [84]:
def compute_accuracy(model, loader):
    pass

def train_model(model, train_loader, val_loader, loss, optimizer, epochs):
    start = time.time()
    
    loss_history = []
    train_istory = []
    val_history  = []
    
    for epoch in range(epochs):
        model.train()
        
        loss_accumulated = 0
        
        i_step = 1
        for (x,y) in enumerate(train_loader):
            optimizer.zero_grad()
            
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            output, hidden = model(x_gpu)
            loss_value = loss(output, y_gpu)
            
            loss_value.backward()
            optimizer.step()
            
            loss_accumulated += loss_value
            
            #val_accuracy = compute_accuracy(model, val_loader)
            print('[{0} @ {4} sec] Loss: {1:4f} Train err: {2:2.2f}% Val err: {3:2.2f}%'. format(
                epoch,
                ave_loss,
                (1 - train_accuracy) * 100,
                (1 - val_accuracy) * 100,
                time.time() - start
            ))
        
        # loss_history.append(float(loss_accum / i_step))
        # val_history.append(val_accuracy)
        # clear_output(wait=True)
        
        i_step += 1
        
    plot_history(loss_history, train_history, val_history)

In [ ]:
lr = 1e-3
epochs = 3
optimizer = optim.Adam(lr = lr)
loss = nn.MSELoss()
model = RNN()

model.to(device)
train_model(model,train_loader, val_loader, loss, optimizer, epochs)

In [ ]:
torch.cuda.empty_cache()
pre
model.eval()
for (x,y) in test_loader:
    with torch.no_grad():
        output = model.forward(x.to(device))
        accuracy = compute_accuracy(model, test_loader)